In [9]:
"""This protocol transfers the main reactant onto the 96 reaction rack. It takes information from 1 csv, stock_reagents. It takes sequentially the reactant from the 24 fluidx racks to the 96 plate"""

from opentrons import robot, containers, instruments
robot.head_speed(x=18000, y=18000, z=5000, a=250, b=250)

class Vector(object):
    def tolist(self):
        return list(self.input_list)

    def astype(self, input_type):
        if input_type == int:
            return Vector([int(float(x)) for x in self.input_list])
        return Vector([input_type(x) for x in self.input_list])

    def __init__(self, input_list):
        self.input_list = input_list


class DataFrame(object):
    def __len__(self):
        return self.length

    def __getitem__(self, value):
        return Vector(self.dict_input[value])

    def __init__(self, dict_input, length):
        self.dict_input = dict_input
        self.length = length


# Function that reads a csv file correctly without having to import anything (issues with molport). Uses 2 classes, Vector and DataFrame
def read_csv(input_file):
    lines = open(input_file).readlines()
    header = lines[0].rstrip().split(",")
    out_d = {}
    for head in header:
        out_d[head] = []
    for line in lines[1:]:
        spl_line = line.rstrip().split(",")
        for i, head in enumerate(header):
            out_d[head].append(spl_line[i])
    df = DataFrame(out_d, len(lines[1:]))
    return df
robot.reset()

# CSV file data
reaction_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\urea_phip_triple_reaction\csv\stock_reagents.csv")

def mainReactant_transfer(reaction):
    # Deck setup
    tiprack_1000 = containers.load("tiprack-1000ul-H", "B3")
    source_trough12row = containers.load("trough-12row", "C1")
    location_stock = containers.load("FluidX_24_5ml", "A1", "int")
    reaction_rack = containers.load("StarLab_96_tall", "D1")
    trash = containers.load("point", "C3")
    # Pipettes SetUp
    p1000 = instruments.Pipette(
        name='eppendorf1000',
        axis='b',
        trash_container=trash,
        tip_racks=[tiprack_1000],
        max_volume=1000,
        min_volume=30,
        channels=1,
    )
    location_header = "Location"
    nb_reaction_header = "Number reaction"
    volume_per_reaction_header = "Volume_stock per reaction"
    volume_max_header = "Volume max per vial"
    nb_reactions = int(reaction_df[nb_reaction_header].tolist()[1])
    volume_per_reaction = int(reaction_df[volume_per_reaction_header].tolist()[1])
    reaction = 0
    """In each well, the volume per reaction of reaction is dispensed, successively in the logical order ("A1", "A2"...). When the maximum amount of reactant in one vial is taken out, the following
    vial in the fluidx rack is used. The transfer stops when the reactant is dispensed in all the wells of the 96 plate or all the wells used for the batch."""
    for index, value in enumerate(reaction_df[location_header].tolist()):
        if reaction < nb_reactions:
            nb_reaction_per_vial = int(float(reaction_df[volume_max_header].tolist()[index]) // nb_reactions)
            source_location = value
            if nb_reactions - reaction < nb_reaction_per_vial:
                if nb_reactions - reaction == 1:
                    p1000.distribute(volume_per_reaction, location_stock.wells(source_location),
                                     reaction_rack.wells(reaction).top())
                else:
                    p1000.distribute(volume_per_reaction, location_stock.wells(source_location),
                                     [x.top() for x in reaction_rack.wells(reaction, to=nb_reactions)])
            else:
                p1000.distribute(volume_per_reaction, location_stock.wells(source_location), [x.top() for x in
                                                                                              reaction_rack.wells(
                                                                                                  reaction,
                                                                                                  to=reaction + nb_reaction_per_vial - 1)])
            reaction = reaction + nb_reaction_per_vial
    robot.home()
mainReactant_transfer(reaction_df)
robot.commands()

KeyError: 'Number reaction'

In [8]:
from opentrons import robot, containers, instruments

robot.head_speed(x=18000, y=18000, z=5000, a=700, b=700)


class Vector(object):
    def tolist(self):
        return list(self.input_list)

    def astype(self, input_type):
        if input_type == int:
            return Vector([int(float(x)) for x in self.input_list])
        return Vector([input_type(x) for x in self.input_list])

    def __init__(self, input_list):
        self.input_list = input_list


class DataFrame(object):
    def __len__(self):
        return self.length

    def __getitem__(self, value):
        return Vector(self.dict_input[value])

    def __init__(self, dict_input, length):
        self.dict_input = dict_input
        self.length = length


# Function that reads a csv file correctly without having to import anything (issues with molport). Uses 2 classes, Vector and DataFrame
def read_csv(input_file):
    lines = open(input_file).readlines()
    header = lines[0].rstrip().split(",")
    out_d = {}
    for head in header:
        out_d[head] = []
    for line in lines[1:]:
        spl_line = line.rstrip().split(",")
        for i, head in enumerate(header):
            out_d[head].append(spl_line[i])
    df = DataFrame(out_d, len(lines[1:]))
    return df


# CSV file data
reactants_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\urea_phip_triple_reaction\csv\amines.csv")
solvent_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\urea_phip_triple_reaction\csv\solvents.csv")
robot.reset()
def reactants_transfer(reactants, solvent):
    # Deck setup

    tiprack_1000 = containers.load("tiprack-1000ul-H", "B3")
    tiprack_1000_2 = containers.load("tiprack-1000ul-H", "D3")
    #tiprack_1000_3 = containers.load("tiprack-1000ul-H", "B1")
    source_trough4row = containers.load("trough-12row", "C2")
    rack_stock_reactants_1 = containers.load("FluidX_24_5ml", "A1", "R_1")
    rack_stock_reactants_2 = containers.load("FluidX_24_5ml", "A2", "R_2")
    rack_stock_reactants_3 = containers.load("FluidX_24_5ml", "B1", "R_3")
    rack_stock_reactants_4 = containers.load("FluidX_24_5ml", "B2", "R_4")
    reaction_rack = containers.load("StarLab_96_tall", "C1")
    trash = containers.load("point", "C3")

    # Pipettes SetUp
    p1000 = instruments.Pipette(
        name='eppendorf1000',
        axis='b',
        trash_container=trash,
        tip_racks=[tiprack_1000, tiprack_1000_2],
        max_volume=1000,
        min_volume=30,
        channels=1,
    )

    location_header = "Location"
    volume_per_reaction_header = "volume_per_vial"
    rack_ID_header = "Rack ID"
    id_header = "CPD ID"
    solvent = "DMA"
    rack_1 = "24_rack1"
    rack_2 = "24_rack2"
    rack_3 = "24_rack3"
    rack_4 = "24_rack4"
    solvent_location_header = "Location_trough"
    solvent_volume_header = "Volume to dispense (uL)"
    counter = 0
    for i, x in enumerate(reactants_df[rack_ID_header].tolist()):
        volume_per_reaction = [reactants_df[volume_per_reaction_header].tolist()[i]]
        source_location = reactants_df[location_header].tolist()[i]

        if x == "":
            #print('null')
            break
        if x == rack_1:
            p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_reactants_1.wells(source_location),
                           reaction_rack.wells(i + 2).top(-5), new_tip='never')
            p1000.drop_tip()
        if x == rack_2:
            p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_reactants_2.wells(source_location),
                           reaction_rack.wells(i + 2).top(-5), new_tip='never')
            p1000.drop_tip()
        if x == rack_3:
            p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_reactants_3.wells(source_location),
                           reaction_rack.wells(i + 2).top(-5), new_tip='never')
            p1000.drop_tip()
        if x == rack_4:
            p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_reactants_4.wells(source_location),
                           reaction_rack.wells(i + 2).top(-5), new_tip='never')
            p1000.drop_tip()
        counter += 1
    # p1000.pick_up_tip()
    for i, x in enumerate(solvent_df[id_header].tolist()):
        if x == solvent:
            vol_to_add = [solvent_df[solvent_volume_header].tolist()[i]]
            solvent_loc = [solvent_df[solvent_location_header].tolist()[i]]

    #p1000.distribute(vol_to_add, source_trough4row.wells(solvent_loc),
     #                [x.top() for x in reaction_rack.wells(0, to=counter + 1)])
    #print(counter)
    robot.home()

reactants_transfer(reactants_df, solvent_df)
robot.commands()

['Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well A1>',
 'Aspirating 440.0 at <Deck><Slot A1><Container R_1><Well A1>',
 'Dispensing 440.0 at <Deck><Slot C1><Container StarLab_96_tall><Well C1>',
 'Drop_tip at <Deck><Slot C3><Container point><Well A1>',
 'Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well B1>',
 'Aspirating 440.0 at <Deck><Slot A1><Container R_1><Well A2>',
 'Dispensing 440.0 at <Deck><Slot C1><Container StarLab_96_tall><Well D1>',
 'Drop_tip at <Deck><Slot C3><Container point><Well A1>',
 'Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well C1>',
 'Aspirating 440.0 at <Deck><Slot A1><Container R_1><Well A3>',
 'Dispensing 440.0 at <Deck><Slot C1><Container StarLab_96_tall><Well E1>',
 'Drop_tip at <Deck><Slot C3><Container point><Well A1>',
 'Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well D1>',
 'Aspirating 440.0 at <Deck><Slot A1><Container R_1><Well A4>',
 'Dispensing 440.0 at <Deck><Slot 

In [5]:
from opentrons import robot, containers, instruments

robot.head_speed(x=18000, y=18000, z=5000, a=700, b=700)


class Vector(object):
    def tolist(self):
        return list(self.input_list)

    def astype(self, input_type):
        if input_type == int:
            return Vector([int(float(x)) for x in self.input_list])
        return Vector([input_type(x) for x in self.input_list])

    def __init__(self, input_list):
        self.input_list = input_list


class DataFrame(object):
    def __len__(self):
        return self.length

    def __getitem__(self, value):
        return Vector(self.dict_input[value])

    def __init__(self, dict_input, length):
        self.dict_input = dict_input
        self.length = length


# Function that reads a csv file correctly without having to import anything (issues with molport). Uses 2 classes, Vector and DataFrame
def read_csv(input_file):
    lines = open(input_file).readlines()
    header = lines[0].rstrip().split(",")
    out_d = {}
    for head in header:
        out_d[head] = []
    for line in lines[1:]:
        spl_line = line.rstrip().split(",")
        for i, head in enumerate(header):
            out_d[head].append(spl_line[i])
    df = DataFrame(out_d, len(lines[1:]))
    return df


# CSV file data
reactants_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\urea_phip_triple_reaction\csv\amines.csv")
solvent_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\urea_phip_triple_reaction\csv\solvents.csv")
robot.reset()
def stock_solution_reactants(reactants, solvent):
    # Deck setup
    tiprack_1000 = containers.load("tiprack-1000ul-H", "B3")
    tiprack_1000_2 = containers.load("tiprack-1000ul-H", "D3")
    source_trough4row = containers.load("trough-12row", "C2")
    rack_stock_reactants_1 = containers.load("FluidX_24_5ml", "A1", "R_1")
    rack_stock_reactants_2 = containers.load("FluidX_24_5ml", "A2", "R_2")
    rack_stock_reactants_3 = containers.load("FluidX_24_5ml", "B1", "R_3")
    rack_stock_reactants_4 = containers.load("FluidX_24_5ml", "B2", "R_4")
    trash = containers.load("point", "C3")
    # Pipettes SetUp
    p1000 = instruments.Pipette(
        name='eppendorf1000',
        axis='b',
        trash_container=trash,
        tip_racks=[tiprack_1000,tiprack_1000_2],
        max_volume=1000,
        min_volume=30,
        channels=1,
    )
    rack_ID_header = "Rack ID"
    id_header = "CPD ID"
    solvent = "DMA"
    rack_1 = "24_rack1"
    rack_2 = "24_rack2"
    rack_3 = "24_rack3"
    rack_4 = "24_rack4"
    location_header = "Location_trough"
    destination_location_header = "Location"
    volume_stock_header = "Volume to dispense (exp) at 0.5M"

    for i, x in enumerate(solvent_df[id_header].tolist()):
        if x == solvent:
            solvent_location = solvent_df[location_header].tolist()[i]
    p1000.pick_up_tip()
    for i, x in enumerate(reactants_df[destination_location_header].tolist()):
        destination_location = x
        vol_to_dispense = [reactants_df[volume_stock_header].tolist()[i]]
        reactants_id = reactants_df[rack_ID_header].tolist()[i]
        if reactants_id == "":
            print ('null')
            break
        #print (rack_ID_header, reactants_id, vol_to_dispense)
        if reactants_id == rack_1:
            print ('rack1')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(solvent_location),
                               rack_stock_reactants_1.wells(destination_location).top(-5), new_tip='never')
        if reactants_id == rack_2:
            print ('rack2')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(solvent_location),
                               rack_stock_reactants_2.wells(destination_location).top(-5), new_tip='never')
        if reactants_id == rack_3:
            print ('rack3')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(solvent_location),
                               rack_stock_reactants_3.wells(destination_location).top(-5), new_tip='never')
        if reactants_id == rack_4:
            print ('rack4')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(solvent_location),
                               rack_stock_reactants_4.wells(destination_location).top(-5), new_tip='never')
    p1000.drop_tip()
    robot.home()
stock_solution_reactants(reactants_df, solvent_df)
robot.commands()

rack1
rack1
rack1
rack1
rack1
rack1
rack1
rack1
rack1
rack1


['Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well A1>',
 'Aspirating 930.6350685 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Dispensing 930.6350685 at <Deck><Slot A1><Container R_1><Well A1>',
 'Aspirating 930.6350685 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Dispensing 930.6350685 at <Deck><Slot A1><Container R_1><Well A1>',
 'Aspirating 946.287964 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Dispensing 946.287964 at <Deck><Slot A1><Container R_1><Well A2>',
 'Aspirating 946.287964 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Dispensing 946.287964 at <Deck><Slot A1><Container R_1><Well A2>',
 'Aspirating 802.543913 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Dispensing 802.543913 at <Deck><Slot A1><Container R_1><Well A3>',
 'Aspirating 802.543913 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Dispensing 802.543913 at <Deck><Slot A1><Container R_1><Well A3>',
 'Aspirating 940.095423 at <Deck><Slo

In [2]:
'''OT1 -Single channel- Making up of main reagent stock solution (2 maximum) using a defined solvent and transfer onto 3mL fluidx vials'''

from opentrons import robot, containers, instruments

robot.head_speed(x=21000, y=21000, z=5000, a=700, b=700)


class Vector(object):
    def tolist(self):
        return list(self.input_list)

    def astype(self, input_type):
        if input_type == int:
            return Vector([int(float(x)) for x in self.input_list])
        return Vector([input_type(x) for x in self.input_list])

    def __init__(self, input_list):
        self.input_list = input_list


class DataFrame(object):
    def __len__(self):
        return self.length

    def __getitem__(self, value):
        return Vector(self.dict_input[value])

    def __init__(self, dict_input, length):
        self.dict_input = dict_input
        self.length = length

robot.reset()
# Function that reads a csv file correctly without having to import anything (issues with molport). Uses 2 classes, Vector and DataFrame
def read_csv(input_file):
    lines = open(input_file).readlines()
    header = lines[0].rstrip().split(",")
    out_d = {}
    for head in header:
        out_d[head] = []
    for line in lines[1:]:
        spl_line = line.rstrip().split(",")
        for i, head in enumerate(header):
            out_d[head].append(spl_line[i])
    df = DataFrame(out_d, len(lines[1:]))
    return df


# CSV file data
stock_reagent_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\urea_phip_triple_reaction\csv\stock_reagents.csv")
solvent_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\urea_phip_triple_reaction\csv\solvents.csv")

"""Function that does two liquid handling transfers. First it will dilute a solid reagent in a big trough to the right concentration.
(Up to 2 reagents). Second, it will transfer the reagent from the big trough to the Fluix 24 vial rack, using the volume from the csv file
It requires 2 csv files. The first is the solvent csv, the second the custom made stock reagent csv."""


def stock_solution(amine, solvent):
    # Deck setup
    tiprack_1000 = containers.load("tiprack-1000ul-H", "B3")
    source_trough4row = containers.load("trough-12row", "C2")
    destination_stock = containers.load("FluidX_24_5ml", "A1", "stock")
    trash = containers.load("point", "C3")
    # Pipettes SetUp
    p1000 = instruments.Pipette(
        name='eppendorf1000',
        axis='b',
        trash_container=trash,
        tip_racks=[tiprack_1000],
        max_volume=1000,
        min_volume=30,
        channels=1,
    )

    id_header = "CPD ID"
    solvent = "DMA"
    stock_sol1 = "stock reagent 1"
    location_header = "Location_trough"
    destination_location_header = "Location"
    volume_stock_header = "Volume to dispense (uL)"
    volume_per_vial = "Volume to dispense"

    for i, x in enumerate(solvent_df[id_header].tolist()):
        if x == solvent:
            solvent_location = solvent_df[location_header].tolist()[i]
        if x == stock_sol1:
            stock_sol1_loc = solvent_df[location_header].tolist()[i]
            stock_sol1_volume = solvent_df[volume_stock_header].tolist()[i]
    # Using the desired solvent, dilution of reagents 1 and/or 2 to the desired conc, in the big trough
    p1000.pick_up_tip()
    p1000.transfer([stock_sol1_volume], source_trough4row.wells(solvent_location),
                   source_trough4row.wells(stock_sol1_loc).top(-5), new_tip='never')
    p1000.drop_tip()

    #robot.pause()

    # Reagents 1 and/or 2 are transfered to the 24 Fluidx vial rack.
    p1000.pick_up_tip()
    for i, x in enumerate(stock_reagent_df[destination_location_header].tolist()):
        destination_location = x
        vol_to_dispense = [stock_reagent_df[volume_per_vial].tolist()[i]]
        stock_id = stock_reagent_df[id_header].tolist()[i]
        if stock_id == stock_sol1:
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(stock_sol1_loc),
                               destination_stock.wells(destination_location).top(-5), new_tip='never')
    p1000.drop_tip()
    robot.home()
stock_solution(stock_reagent_df, solvent_df)
robot.commands()

['Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well A1>',
 'Aspirating 1000.0 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Dispensing 1000.0 at <Deck><Slot C2><Container trough-12row><Well A2>',
 'Aspirating 1000.0 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Dispensing 1000.0 at <Deck><Slot C2><Container trough-12row><Well A2>',
 'Aspirating 1000.0 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Dispensing 1000.0 at <Deck><Slot C2><Container trough-12row><Well A2>',
 'Aspirating 789.0 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Dispensing 789.0 at <Deck><Slot C2><Container trough-12row><Well A2>',
 'Aspirating 789.0 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Dispensing 789.0 at <Deck><Slot C2><Container trough-12row><Well A2>',
 'Drop_tip at <Deck><Slot C3><Container point><Well A1>',
 'Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well B1>',
 'Aspirating 1000.0 at <Deck><Slot C2><Container tro

In [ ]:
    # Using the desired solvent, dilution of reagents 1 and/or 2 to the desired conc, in the big trough
    p1000.pick_up_tip()
    p1000.transfer([stock_sol1_volume], source_trough4row.wells(solvent_location),
                   source_trough4row.wells(stock_sol1_loc).top(-5), new_tip='never')
    p1000.drop_tip()

    robot.pause()

    # Reagents 1 and/or 2 are transfered to the 24 Fluidx vial rack.
    for i, x in enumerate(stock_reagent_df[destination_location_header].tolist()):
        destination_location = x
        vol_to_dispense = [stock_reagent_df[volume_per_vial].tolist()[i]]
        stock_id = stock_reagent_df[id_header].tolist()[i]
        if stock_id == stock_sol1:
            p1000.pick_up_tip()
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(stock_sol1_loc),
                               destination_stock.wells(destination_location).top(-5), new_tip='never')
            p1000.drop_tip()
    robot.home()
stock_solution(stock_reagent_df, solvent_df)
robot.commands()